# EXPLORATORY DATA ANALYSIS - SISTEMA COMANDO

### FOCUS ON INCIDENTS CAUSED BY METHEOROLOGICAL EVENTS

---
### Notebook Sections:
1. Exploratory data analysis & data cleaning
    * Asses general quality of the dataset
2. Extract incident catalog
    * Extract catalog of incidents with known coordinates, caused by rain for predicive modeling
3. Reverse geocode catalog coordinates
4. Reverse geocode result analysis
5. Filter and merge geocode result to catalog
    * Obs: Exclude incidents without a numbered address result
4. Extra: Search in text based records for incidents left out of initial extraction 
5. Extra: Search catalog for incidents indentified by city cameras

---

### Import modules and functions

In [1]:
import os, pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns; sns.set()
from IPython.display import clear_output as co

class DATA:
    path = r'C:\Users\luisr\Desktop\Repositories\Dados\Desafio COR-Rio IV\\'
    AlertaAPI = r'http://websempre.rio.rj.gov.br/json/chuvas'

### Load data

In [16]:
comando = pd.read_csv(DATA.path + 'comando.csv')

---
# 1. Data Cleaning & Exploratory Data Analysis

### 0. Extract incidents caused by metheorological events

#### Incidents categories

In [3]:
titles = [
    "Bolsão d'água em via", 'Vazamento de água / esgoto',
    'Alagamentos e enchentes', "Lâmina d'água",
    "Lâmina d'água em via", 'Alagamento',
    'Enchente', 'Bueiro'
]

#### Extract records

In [17]:
records = comando[comando['POP_TITULO'].isin(titles)]
records_left = comando.loc[set(comando.index).difference(records.index)]

data = records.reset_index(drop=False).rename(columns={'index': 'REGISTRO_ID'}); comando = None

records.shape, records_left.shape # Extracted and left records shapes

((12409, 18), (192732, 18))

**Key findings**:
1. The record history of incidents caused by rain has some overlaping categories for incident type. There is a need to confirm the reason for the overlaping categories with the institution.

### 1. Asses columns and data types

##### Identify variables' types

In [18]:
cols = {
    'text': [
        'EVENTO_TITULO', 'EVENTO_DESCRICAO'
    ],
    'categorical': [
        'EVENTO_GRAVIDADE', 'EVENTO_BAIRRO', 'STATUS', 'EVENTO_PRAZO',
        'POP_TITULO', 'POP_DESCRICAO', 'ORGAO_SIGLA', 'ORGAO_NOME', 'ACAO'
    ],
    'datetime': [
        'EVENTO_INICIO', 'EVENTO_INICIO_HORA',
        'EVENTO_FIM', 'EVENTO_FIM_HORA'
    ],
    'location': [
        'EVENTO_LATITUDE', 'EVENTO_LONGITUDE'
    ]
}

#### Check raw data types

In [19]:
data.dtypes

REGISTRO_ID             int64
EVENTO_ID               int64
EVENTO_TITULO          object
EVENTO_DESCRICAO       object
EVENTO_GRAVIDADE       object
EVENTO_BAIRRO          object
STATUS                 object
EVENTO_INICIO          object
EVENTO_INICIO_HORA     object
EVENTO_FIM             object
EVENTO_FIM_HORA        object
EVENTO_PRAZO           object
EVENTO_LATITUDE       float64
EVENTO_LONGITUDE      float64
POP_TITULO             object
POP_DESCRICAO          object
ORGAO_SIGLA            object
ORGAO_NOME             object
ACAO                   object
dtype: object

---
### 2. Asses time variables

#### Convert event start and end dates and times to 'datetime64' type

In [20]:
start = data['EVENTO_INICIO'] + ' ' + data['EVENTO_INICIO_HORA']
end = data['EVENTO_FIM'] + ' ' + data['EVENTO_FIM_HORA']

data['evento_inicio'] = pd.to_datetime(start)
data['evento_fim'] = pd.to_datetime(end)

data['evento_inicio']

0       2015-04-10 15:59:00
1       2015-04-10 15:59:00
2       2015-04-10 15:59:00
3       2015-04-10 15:59:00
4       2015-05-05 07:52:00
                ...        
12404   2022-04-30 12:33:00
12405   2022-04-30 15:33:00
12406   2022-04-30 15:33:00
12407   2022-04-30 16:09:00
12408   2022-04-30 16:09:00
Name: evento_inicio, Length: 12409, dtype: datetime64[ns]

---
### 2. Asses text variables

#### Unique values count

In [21]:
for col in cols['text']: # descomente para conferir contagem de valores unicos
    display(data[col].value_counts().to_frame(col))

,EVENTO_TITULO
Bolsão d'água em via,3959
Bolsão d'água,1752
Alagamento,1112
Vazamento de água,568
Bolsão d'água,554
...,...
Limpeza da Via,1
Bolsão d'água em via ( CAM 294 ),1
Lamina d'água ( CAM 136 ),1
Manutenção da Via ( Seconserva ),1


,EVENTO_DESCRICAO
"Rua do Catete, Alt. R. Silveira Martins - Catete",67
"R. São Clemente, 226 - Botafogo",31
Mergulhão Billy Blanco - Barra da Tijuca,25
Av. Brasil - Benfica,25
"Av. Epitácio Pessoa, Alt. R. Maria Quitéria - Lagoa",24
...,...
Av. Brasil - Bangu - Sent. Centro,1
"Rua Uçá, alt. Praça Jerusalém - Jardim Guanabara",1
"Av. Radial Oeste, 173 - Praça da Bandeira",1
R. Humaitá - Alt. R. Jardim Botânico,1


Conclusions:

Only datetime variables require type conversion.

---
### 3. Asses categorical variables

Categorical columns (unique categories):
* EVENTO_GRAVIDADE (BAIXO, MEDIO, ALTO, CRITICO, SEM_CLASSIFICAÇÃO). Extra values: Litoral do Rio de Janeiro
* EVENTO_BAIRRO (339) - Desestruturado
    * Contém 'nan'
    * Valores de categorias não uniforme: Bairros de mesmo nome com escrita diferente.
    * Contem valores inválidos: Nomes de rua, números, endereços, etc.
    * Necessita método strip()
* EVENTO_STATUS (FECHADO, ABERTO). Extra values: FINALIZADO, Litoral do Rio de Janeiro
* EVENTO_PRAZO (CURTO, MEDIO, LONGO)
* POP_TITULO (40)
* POP_DESCRICAO (42)
* ORGAO_SIGLA (34)
* ORGAO_NOME (35)
* ACAO (70)

In [22]:
for col in cols['categorical']: # descomente para conferir contagem de valores unicos
    display(data[col].value_counts().to_frame(col))

,EVENTO_GRAVIDADE
BAIXO,11175
MEDIO,953
ALTO,217
CRITICO,59
SEM_CLASSIFICACAO,4


,EVENTO_BAIRRO
Barra da Tijuca,671
Lagoa,364
Centro,323
Botafogo,239
Itanhangá,228
...,...
Tubiacanga,1
Engenho de Dentro,1
Catumbi,1
Oswaldo Cruz,1


,STATUS
FECHADO,12380
ABERTO,27


,EVENTO_PRAZO
CURTO,6755
MEDIO,82
LONGO,51


,POP_TITULO
Bolsão d'água em via,8956
Vazamento de água / esgoto,1707
Alagamento,821
Alagamentos e enchentes,574
Lâmina d'água,260
Bueiro,75
Enchente,16


,POP_DESCRICAO
Bolsão d'água em via,8956
Vazamento de água / esgoto,1707
Alagamento,821
Alagamentos e enchentes,574
Lâmina d'água em via,260
Bueiro,75
Enchente,16


,ORGAO_SIGLA
COMLURB,4346
SECONSERVA,3386
CET-RIO,2301
CEDAE,907
PortoNovo,460
GM,357
LAMSA,245
BOMBEIROS,105
DefesaCivil,99
Light,48


,ORGAO_NOME
Companhia de Limpeza Urbana,4346
Secretaria Municipal de Conservação,3386
Companhia de Engenharia de Tráfego,2301
Companhia Estadual de Águas e Esgotos,907
Porto Novo S.A.,460
Empresa Municipal de Vigilância,357
Linha Amarela S.A.,245
Corpo de Bombeiros,105
Secretaria Municipal de Defesa Civil,99
Light Serviços de Eletricidade S.A.,48


,ACAO
Desobstruir ralos e bueiros,3754
Drenar água,3247
Organizar o trânsito,2101
Desfazer o acidente,1195
Fazer limpeza do local,572
Consertar vazamento de água,497
Consertar vazamento de esgoto,404
Prevenir perigo,108
Resgatar pessoas,80
Monitorar possíveis interdições,69


---
### 4. Asses location variables

#### Functions to process coordinates

#### Correct magnitude and replace with NAN coordinates below decimal precision of four places.

Obs: This method aims at excluding below defined precision but it 2also excludes values with precision equal to the defined one but ending with zero (1/10 of the values). For example, 22.9430 is excluded even tough is 4 places precise.

In [ ]:
from Modules.decimals import correctMagnitude, dropBelowPrecision

In [24]:
# Correct magnitude of coordinates
data['lat'] = correctMagnitude(data['EVENTO_LATITUDE'], mag=1)
data['lng'] = correctMagnitude(data['EVENTO_LONGITUDE'], mag=1)

# Fill coordinates below decimal precision of 4 places
min_precision = 4

data[['lat', 'lng']] = fillNadropBelowPrecision(
    data[['lat', 'lng']],
    decimal=min_precision,
    subset='all'
)

<ipython-input-23-d2094f930cb3>:4: RuntimeWarning: divide by zero encountered in log10
  return np.floor(np.log10(abs(number)))
<ipython-input-23-d2094f930cb3>:9: RuntimeWarning: invalid value encountered in double_scalars
  return number / 10 ** ( orderOfMagnitude(number) - mag )
c:\Users\luisr\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-23-d2094f930cb3>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[below_msk] = np.nan


#### Percentage of records without valid coordinates

In [25]:
data['lat'].isna().sum() / len(data) * 100 # 6 % dos incidentes sem cordenadas.

19.84849705858651

In [26]:
data.groupby('EVENTO_ID').first()['lat'].isna().sum() / len(data) * 100 # 16 % dos registros sem cordenadas.

7.196389717140785

---
# 6. EXTRA: Term based search in text records for incidents left out of catalog

#### Function to search for terms in pandas series of type string

In [60]:
def drop_accents(string):
    item = string.lower()
    for a, b in zip(
        ['á', 'ã', 'â', 'é', 'í', 'ó', 'õ', 'ú', 'ç'],
        ['a', 'a', 'a', 'e', 'i', 'o', 'o', 'u', 'c']
    ):
        item = item.replace(a, b)
    return item

def search_records(records, queries, avoid=[], report_interval=10000): # accepts records argument as pandas series
    matches = []
    for i, index in enumerate(records.index):
        if i%report_interval == 0: print(f'{i+1}/{records.shape[0]} records searched'); co(wait=True)
        record = records.loc[index]
        if type(record)==str:
            record = drop_accents(record)
            query_matches, match_words = [], []
            for query in queries:
                if drop_accents(query) in record:
                    words = record.split(' ')
                    for word in words:
                        if query in word:
                            if word not in avoid:
                                match_words.append(word)
                    if match_words:
                        query_matches.append(query)
            if query_matches:
                matches.append([index, query_matches, match_words])
    print(f'Done! {len(records)} terms searched.')
    return pd.DataFrame(matches, columns=['index', 'matched_queries', 'matched_words'])

### Term search in text records for incidents left out of catalog

In [61]:
queries = [
    'bolsao', 'bolsoes', 'alagamento',
    'lamina', 'espelho', 'enchente',
    'inundacão', 'vazamento', 'bueiro',
    'rompimento', 'adutora', 'tubulação'
]

term_search = search_records(records_left['EVENTO_TITULO'], queries, avoid=[]) # searching in records left out of initial extraction

Done! 192732 terms searched.


In [62]:
term_search['matched_queries'].value_counts().to_frame()

,matched_queries
[bueiro],404
[vazamento],307
"[rompimento, tubulação]",12
[alagamento],11
[adutora],7
"[vazamento, bueiro]",5
[bolsao],1


---
# 7. Extra: Searching incidents identified by city cameras

#### Search for 'cam' term in text records

In [63]:
cam_search = search_records(full_catalog['EVENTO_TITULO'], queries=['cam'])

cam_search['matched_words'].value_counts()

Done! 3987 terms searched.


[cam]         142
[(cam]         72
[(camera]      58
[cam.]          6
[(cam.]         5
[camera]        1
[(cam278)]      1
[(cam48)]       1
[(cam269)]      1
[(cam145)]      1
[(cam107)]      1
[(cam116)]      1
[(cam147)]      1
[(cam262)]      1
[(cam430)]      1
[(cam298)]      1
[(cam354)]      1
[(cam243)]      1
[(cam112)]      1
Name: matched_words, dtype: int64

#### Percentage of incidents identified by camera

In [64]:
cam_records = full_catalog.loc[cam_search['index']]

print(round(cam_records.shape[0] / full_catalog.shape[0] * 100, 2), '%')

7.45 %


In [65]:
cam_records['POP_TITULO'].value_counts().to_frame()

,POP_TITULO
Bolsão d'água em via,263
Vazamento de água / esgoto,14
Alagamento,9
Lâmina d'água,8
Bueiro,3
